# 📌 Objetivo del prototipo

**Enviar notificaciones push a una app móvil Flutter (Android) desde un endpoint backend.**

### Componentes del sistema

- **`mero_notification`**: App móvil Flutter (Android) que recibe notificaciones push
- **`mero_backend`**: Backend NestJS con endpoint `POST /notify` para enviar notificaciones
- **`mero_admin_panel`**: Panel admin Flutter Desktop (opcional, para envío manual desde GUI)

### Stack tecnológico

- **Firebase Cloud Messaging v1 API** (no legacy)
- **Flutter**: `firebase_core` + `firebase_messaging`
- **Node.js**: `firebase-admin` SDK
- **NestJS** + TypeScript + Swagger UI

### Flujo general

```
Backend (NestJS) → Firebase Cloud Messaging → App móvil (Flutter)
```

---

**⚠️ Limitación crítica**: Las notificaciones push **NO funcionan en Flutter Web** ni en emuladores sin Google Play Services. Solo en **Android con Google Play Services** o dispositivos físicos.


# 🔧 Paso 1: Configuración en Firebase Console

### 1.1 Crear proyecto Firebase

1. Ve a [console.firebase.google.com](https://console.firebase.google.com)
2. Crea un nuevo proyecto (o usa uno existente)
3. Anota los siguientes datos:
   - **Project ID**: `meronotification`
   - **Project Number (Sender ID)**: `121------293`
   - **Estos 2 datos estan en la configuracion general de su proyecto de firebase**

### 1.2 Registrar app Android

1. NewAppFlutter >"Descripción general del proyecto" > **Agregar app** > **Android**
2. Package name: `com.example.mero_notification` (debe coincidir exactamente con el `applicationId` de Gradle)  
3. Descarga el archivo **`google-services.json`  (firebase console > configuracion del proyecto > cuentas de servicio > SDK de Firebase Admin > Generar nueva clave privada)**
4. **Ubicación final**: `mero_notification/android/app/google-services.json`

### 1.3 Generar clave de cuenta de servicio (para el backend)

1. Ve a **Configuración del proyecto** > **Cuentas de servicio**
2. Haz clic en **"Generar nueva clave privada"**
3. Descarga el archivo **`serviceAccountKey.json`** (Trae tus ID's del proyecto firebase)
4. **Ubicación final**: `mero_backend/src/config/serviceAccountKey.json`

---

### 🔒 Seguridad

**❌ NUNCA** subas `serviceAccountKey.json` a Git. Este archivo permite enviar notificaciones en nombre de tu proyecto. Agrégalo a `.gitignore`.

---

### Archivos obtenidos

| Archivo | Ubicación | Propósito |
|---------|-----------|----------|
| `google-services.json` | `mero_notification/android/app/` | Configuración de Firebase en la app móvil |
| `serviceAccountKey.json` | `mero_backend/src/config/` | Credenciales para enviar notificaciones desde el backend |


# 📱 Paso 2: App móvil (`mero_notification`)

### 2.1 Agregar dependencias

En `pubspec.yaml`:

```yaml
dependencies:
  flutter:
    sdk: flutter
  firebase_core: ^2.27.0
  firebase_messaging: ^14.7.10
```

Ejecutar: `flutter pub get`

---

### 2.2 Configurar Gradle

#### `android/build.gradle.kts` (o `settings.gradle.kts`):

```kotlin
plugins {
    id("com.google.gms.google-services") version "4.4.3" apply false
}
```

#### `android/app/build.gradle.kts`:

```kotlin
plugins {
    id("com.android.application")
    id("kotlin-android")
    id("dev.flutter.flutter-gradle-plugin")
    id("com.google.gms.google-services")  // ← Agregar esta línea
}

android {
    namespace = "com.example.mero_notification"
    defaultConfig {
        applicationId = "com.example.mero_notification"  // ← Debe coincidir con Firebase
        minSdk = 21  // ← Requerido por FCM
    }
}
```

---

### 2.3 Colocar `google-services.json`

**Ubicación exacta**: `mero_notification/android/app/google-services.json`

Este archivo es procesado automáticamente por el plugin de Google Services durante la compilación.

---

### 2.4 Agregar permiso de notificaciones (Android 13+)

En `android/app/src/main/AndroidManifest.xml`:

```xml
<manifest xmlns:android="http://schemas.android.com/apk/res/android">
    <uses-permission android:name="android.permission.POST_NOTIFICATIONS" />
    <!-- resto del contenido -->
</manifest>
```


# 📱 Paso 2 (continuación): Implementación en `lib/main.dart`

### 2.5 Código principal de la app

#### Inicialización de Firebase:

```dart
Future<void> main() async {
  WidgetsFlutterBinding.ensureInitialized();
  await Firebase.initializeApp();  // ← Inicializa Firebase
  
  FirebaseMessaging.onBackgroundMessage(_firebaseMessagingBackgroundHandler);
  
  // Solicita permisos de notificación
  await FirebaseMessaging.instance.requestPermission();
  
  runApp(const MyApp());
}

// Handler para notificaciones en segundo plano
Future<void> _firebaseMessagingBackgroundHandler(RemoteMessage message) async {
  await Firebase.initializeApp();
}
```

---

#### Obtener token FCM:

```dart
final token = await FirebaseMessaging.instance.getToken();
print('Token FCM: $token');
```

**Este token** es el identificador único del dispositivo. Se debe enviar al backend para que pueda enviar notificaciones a este dispositivo específico.

---

#### Escuchar notificaciones en primer plano:

```dart
FirebaseMessaging.onMessage.listen((RemoteMessage message) {
  print('Mensaje recibido en foreground: ${message.notification?.title}');
  // Mostrar SnackBar o diálogo
});
```

#### Escuchar cuando el usuario abre una notificación:

```dart
FirebaseMessaging.onMessageOpenedApp.listen((RemoteMessage message) {
  print('Usuario abrió la notificación');
  // Navegar a pantalla específica
});
```

---

### Comportamiento de las notificaciones

| Situación | Comportamiento |
|-----------|---------------|
| **App en segundo plano o cerrada** | Notificación del sistema en la bandeja de Android |
| **App en primer plano (abierta)** | Se ejecuta `onMessage`, mostramos `SnackBar` manualmente |
| **Usuario toca la notificación** | Se ejecuta `onMessageOpenedApp` |


# ⚙️ Paso 3: Backend (`mero_backend`)

### Este backend para las notification push esta pensado en hacer un enpoint para un admin_panel (el cual mi cliente envia notificaciones sus usuarios por medio de la app), pero si no quieres tener un backend y todo lo anterior te funciona se puede hacer pruebas de notifications desde FireBaseConsole > Messaging > Crear primer campana > firebase notification > minimo llenar los campos: titulo, textoNotificacion, orientacion: app - mero_notification (android) > guardar como borrador > abrir el borrador > publicar.
### Tiempo: SIN backend las notification por firebase tienen un tiempo estimado de 30-90sg en hacerse visible en el emulador (depende si la app esta en segundo plano o cerrado), CON backend usando un enpoint la respuesta de la notification es casi intantanea <5sg en el emulador

### 3.1 Instalar dependencias

```bash
npm install firebase-admin class-validator class-transformer
npm install @nestjs/swagger swagger-ui-express
```

---

### 3.2 Colocar `serviceAccountKey.json`

**Ubicación**: `mero_backend/src/config/serviceAccountKey.json`

---

### 3.3 Implementación del servicio FCM

**Archivo**: `src/fcm/fcm.service.ts`

```typescript
import * as admin from 'firebase-admin';
import { join } from 'path';

// Tokens hardcodeados para pruebas (en producción usar DB)
// Al completar el backend el movil q abra la app le da un token FCM, copialo y colocalo en la listTokenFcm 
const TARGET_TOKENS: string[] = [
  'cMGoA3WXTh2rS532B2fmzd:APA91bEvXh-QmShOqR7Qa...',  // ← Aqui colocar el token q les da la ap en el emulador/movil
];

@Injectable()
export class FcmService {
  private initialized = false;

  private ensureInitialized() {
    if (this.initialized) return;
    const serviceAccountPath = join(process.cwd(), 'src', 'config', 'serviceAccountKey.json');
    admin.initializeApp({
      credential: admin.credential.cert(serviceAccountPath),
    });
    this.initialized = true;
  }

  async sendToTargets(title: string, body: string) {
    this.ensureInitialized();
    const response = await admin.messaging().sendEachForMulticast({
      tokens: TARGET_TOKENS,
      notification: { title, body },
    });
    return response;
  }
}
```

**Nota**: `sendEachForMulticast()` devuelve información detallada de éxito/falla por cada token.

---

### 3.4 Endpoint del controlador

**Archivo**: `src/fcm/fcm.controller.ts`

```typescript
@Controller('notify')
export class FcmController {
  constructor(private readonly fcmService: FcmService) {}

  @Post()
  async notify(@Body() dto: NotifyDto) {
    const result = await this.fcmService.sendToTargets(dto.title, dto.body);
    return {
      successCount: result.successCount,
      failureCount: result.failureCount,
      failures: result.responses.filter(r => !r.success).length,
    };
  }
}
```

**Endpoint**: `POST http://localhost:3000/notify`

**Body**:
```json
{
  "title": "Título de la notificación",
  "body": "Cuerpo del mensaje"
}
```


# ⚙️ Paso 3 (continuación): Swagger y ejecución

### 3.5 Configurar Swagger

**Archivo**: `src/main.ts`

```typescript
import { NestFactory } from '@nestjs/core';
import { SwaggerModule, DocumentBuilder } from '@nestjs/swagger';
import { AppModule } from './app.module';

async function bootstrap() {
  const app = await NestFactory.create(AppModule);
  
  const config = new DocumentBuilder()
    .setTitle('FCM Notification API')
    .setVersion('1.0')
    .build();
  const document = SwaggerModule.createDocument(app, config);
  SwaggerModule.setup('api', app, document);
  
  await app.listen(3000);
}
bootstrap();
```

**Swagger UI**: `http://localhost:3000/api`

---

### 3.6 Ejecutar el backend

```bash
cd mero_backend
npm install
npm run start:dev
```

Salida esperada:
```
[Nest] LOG [NestApplication] Nest application successfully started
```

---

### Flujo interno del backend

```
1. Cliente envía POST /notify con {title, body}
2. FcmController valida el DTO
3. FcmService inicializa firebase-admin (primera vez)
4. Llama a admin.messaging().sendEachForMulticast()
5. Firebase Cloud Messaging procesa el envío
6. Responde con {successCount, failureCount, failures}
```

---

**⚡ Rendimiento**: Las notificaciones llegan en **< 2 segundos** cuando se envían desde la API (flujo de producción directo).


# 📤 Paso 4: Prueba del flujo completo

### 4.1 Ejecutar la app móvil

```bash
cd mero_notification
flutter pub get
emuladorOn
flutter run
```

**Requisitos**:
- Usar emulador de Android Studio **con Google Play Services** (no "Vanilla")
- O usar un dispositivo físico Android

**Resultado esperado**:
- La app muestra el **token FCM** en pantalla
- Ejemplo: `cMGoA3WXTh2rS532B2fmzd:APA91bEvXh-QmShOqR7QaIBOp1...`

---

### 4.2 Copiar el token FCM

1. Selecciona y copia el token completo desde la app
2. Abre `mero_backend/src/fcm/fcm.service.ts`
3. Pega el token en el array `TARGET_TOKENS`:

```typescript
const TARGET_TOKENS: string[] = [
  'cMGoA3WXTh2;2B2fmzd:APA91bEvXh-QmShOqR7QaIBOp1...',  // ← Token real
];
```

4. Guarda el archivo (el backend se recargará automáticamente con `ts-node-dev`)

---

### 4.3 Enviar notificación desde Swagger

1. Abre `http://localhost:3000/api` en el navegador
2. Expande el endpoint **`POST /notify`**
3. Haz clic en **"Try it out"**
4. Ingresa el payload:

```json
{
  "title": "Prueba desde Swagger",
  "body": "Si ves esto, FCM está funcionando correctamente"
}
```

5. Haz clic en **"Execute"**

---

### 4.4 Verificar la notificación

**Caso 1: App en segundo plano o cerrada**
- Minimiza o cierra la app móvil
- Envía la notificación desde Swagger
- ✅ **Resultado**: Notificación aparece en la bandeja de Android en < 2 segundos

**Caso 2: App en primer plano (abierta)**
- Deja la app abierta
- Envía la notificación
- ✅ **Resultado**: Aparece un `SnackBar` con el título y cuerpo

---

### Respuesta esperada del backend

```json
{
  "successCount": 1,
  "failureCount": 0,
  "failures": 0
}
```

**Si `successCount: 1`** → ✅ La notificación se envió exitosamente  
**Si `failureCount: 1`** → ❌ Token inválido, expirado o incorrecto


# ⚠️ Notas críticas (¡NO omitir!)

### 1. Las notificaciones NO aparecen en Firebase Console si se envían desde la API

**Comportamiento normal**: Cuando envías notificaciones usando `firebase-admin` (desde tu backend), **NO verás el envío en Firebase Console**.

Según la [documentación oficial de FCM](https://firebase.google.com/docs/cloud-messaging/send-message):
> *"Las notificaciones enviadas mediante la API administrativa no se registran en la consola de Firebase. Solo se registran las enviadas desde la interfaz de usuario de Firebase Console."*

**Esto es completamente normal y esperado.**

---

### 2. No funciona en Flutter Web

**FCM para web** requiere:
- Service Workers
- Configuración de VAPID keys
- Un flujo completamente diferente

Este prototipo **solo funciona en Android con Google Play Services**.

---

### 3. Tú debes gestionar tokens y registro de dispositivos

**Firebase NO almacena ni gestiona tokens automáticamente.** Eres tú quien debe:

- Obtener el token en la app: `await FirebaseMessaging.instance.getToken()`
- Enviar el token a tu backend (vía API REST, por ejemplo)
- Almacenar el token en tu base de datos asociado al usuario/dispositivo
- Usar el token cuando quieras enviar notificaciones

**Gestión de tokens expirados**:
```dart
FirebaseMessaging.instance.onTokenRefresh.listen((newToken) {
  // Enviar el nuevo token al backend
});
```

---

### 4. Diferencia entre flujos de envío

| Método de envío | Latencia | Uso |
|----------------|----------|-----|
| **Firebase Console** ("Envía un mensaje de prueba") | 30-60 segundos | Solo para pruebas manuales (usa cola de prueba) |
| **API v1** (firebase-admin) | < 2 segundos | Producción (envío directo) |

Según la [guía oficial "Envía un mensaje de prueba a una app en segundo plano"](https://firebase.google.com/docs/cloud-messaging/flutter/first-message):
> *"Los mensajes de prueba enviados desde Firebase Console pueden tardar hasta 60 segundos en llegar, ya que utilizan una cola de menor prioridad."*

**Por eso este prototipo usa la API v1**: para obtener entregas instantáneas en producción.


# ✅ Validación final

### Checklist de verificación

| Criterio | ✅ Correcto | ❌ Incorrecto |
|----------|------------|-------------|
| **Notificación llega en < 2 segundos** | Usando API v1 (producción) | Usando Firebase Console (prueba) |
| **Respuesta del backend** | `successCount: 1, failureCount: 0` | `successCount: 0, failureCount: 1` |
| **Notificación en segundo plano** | Aparece en bandeja de Android | No aparece o da error |
| **Notificación en primer plano** | Muestra SnackBar (código implementado) | No pasa nada (falta `onMessage` listener) |
| **Token FCM** | Se muestra en la app al iniciar | "Obteniendo token..." indefinidamente |

---

### Confirmación de que todo funciona

**Si la notificación llega en < 2 segundos cuando la app está en segundo plano** → ✅ **Todo está correcto**.

Esto confirma que:
1. Firebase está correctamente configurado en la app
2. El backend tiene las credenciales correctas
3. El token FCM es válido
4. Estás usando el **flujo de producción de FCM v1 API**
5. Google Play Services está funcionando en el dispositivo

---

### Errores comunes y soluciones

| Problema | Causa | Solución |
|----------|-------|----------|
| `failureCount: 1` | Token inválido o expirado | Copiar token actual de la app y pegarlo en `fcm.service.ts` |
| No llega notificación | Emulador sin Google Play Services | Usar emulador "with Google Play" o dispositivo físico |
| App no muestra token | `google-services.json` incorrecto o `applicationId` no coincide | Verificar que el package name coincida exactamente |
| Error al inicializar Firebase Admin | `serviceAccountKey.json` falta o es inválido | Descargar nueva clave desde Firebase Console |

---

### Próximos pasos para producción

1. **Base de datos para tokens**: Reemplazar array `TARGET_TOKENS` por consulta a PostgreSQL/MongoDB
2. **Autenticación**: Agregar JWT o API keys al endpoint `/notify`
3. **Manejo de errores**: Implementar retry logic y limpieza de tokens expirados
4. **Notificaciones ricas**: Agregar `flutter_local_notifications` para iconos, sonidos y acciones
5. **Segmentación**: Usar Firebase Topics para envíos masivos sin gestionar tokens individuales
6. **Variables de entorno**: Mover `serviceAccountKey.json` a variables de entorno (ej: `process.env.FIREBASE_CREDS`)
